In [1]:
import production as p
import os
import inspect
import sys
import pandas as pd

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Step1: Read the LLM-ready data

In [36]:
df_user_data,neutral_sentences = p.read_input_data()

In [37]:
df_user_data.head(3)

,username,messageID,original
49,U0,29,"I have a younger sister, 22 now, and a mother ..."
47,U0,20,I am gonna sound very young when I tell you ab...
46,U0,2,So there is one that we have in common and tha...


In [38]:
neutral_sentences

,idSentence,sentences
0,0,I’m all about that food. I usually kick off th...
1,1,Just getting my vitamins in at the school cant...
2,2,De Pizzabakkers sell this vegan pizza with che...
3,3,I entered the world of vegan foods lately. Nex...
4,4,This vegan fried chicken from KFC is on the sp...
5,5,Just having this vegan hotdog from the school ...
6,6,"Just had my first vegan cake at ""groene bakker..."
7,7,This vegan chocolate is on point. Its with oat...
8,8,I'm all about fast food. I'm having some fries...
9,9,"I went to the ""Groene Burger"" fast food restau..."


### Step 2 LLM TST

In [101]:
# This calls Mistral API
df_llm_tst = p.llm_tst(df_user_data,neutral_sentences[0:2])
df_llm_tst_final = p.postprocess_llm_tst(df_llm_tst)
df_llm_tst_final.head(3)

In [7]:
# This gets the LLM TST data already generated
df_llm_tst_final = p.read_and_postprocess_llm_tst()

from datetime import datetime
smallest_timestamp = datetime.strptime(str(df_llm_tst_final['timestamp'].min()), "%Y%m%d%H%M%S").strftime("%B %d %H:%M:%S")

largest_timestamp = datetime.strptime(str(df_llm_tst_final['timestamp'].max()), "%Y%m%d%H%M%S").strftime("%B %d %H:%M:%S")
print(smallest_timestamp,largest_timestamp)

May 20 12:02:53 May 24 19:05:49


### Step 3 LLM Evaluate

In [108]:
df_llm_tst_final

,username,id_neutral_sentence,neutral_sentence,tst_id,tst_sentence,explanation,llm_tst,query,model,prompt_tokens,completion_tokens,object,promptID,timestamp
0,U0,0,I’m all about that food. I usually kick off th...,U020240520120253,"I'm totally into food, you know? I start my we...","In the style of person X, the sentence is mor...","tst\_sentence: ""I'm totally into food, you kno...",\n[INST]You are an expert in text style transf...,mistral-small,325,86,chat.completion,2,20240520120253
1,U0,1,Just getting my vitamins in at the school cant...,U020240520120254,So I'm just getting my vitamins at the school ...,"In this conversational style of person X, cas...","tst\_sentence: ""So I'm just getting my vitamin...",\n[INST]You are an expert in text style transf...,mistral-small,329,119,chat.completion,2,20240520120254
2,U1,0,I’m all about that food. I usually kick off th...,U120240520120258,"Got to have my food fix, you know? Startin' th...",Person X uses a conversational style that is ...,"tst\_sentence: ""Got to have my food fix, you k...",\n[INST]You are an expert in text style transf...,mistral-small,322,111,chat.completion,2,20240520120258
3,U1,1,Just getting my vitamins in at the school cant...,U120240520120259,"No worries about my math exam, I'm good to go!...","In the style of person X, the response is mor...","tst\_sentence: ""No worries about my math exam,...",\n[INST]You are an expert in text style transf...,mistral-small,326,124,chat.completion,2,20240520120259
4,U2,0,I’m all about that food. I usually kick off th...,U220240520120304,You know what I'm totally into? Food! I start ...,"In this conversational style of person X, I a...","tst\_sentence: ""You know what I'm totally into...",\n[INST]You are an expert in text style transf...,mistral-small,290,166,chat.completion,2,20240520120304
5,U2,1,Just getting my vitamins in at the school cant...,U220240520120306,"Hey, just grabbing some grub at the school caf...","In this conversational style of person X, the...","tst\_sentence: ""Hey, just grabbing some grub a...",\n[INST]You are an expert in text style transf...,mistral-small,294,157,chat.completion,2,20240520120306
6,U3,0,I’m all about that food. I usually kick off th...,U320240520120310,"Yo, got to get my weekly dose of that vegan go...","Person X uses a more casual and upbeat tone, ...","tst\_sentence: ""Yo, got to get my weekly dose ...",\n[INST]You are an expert in text style transf...,mistral-small,319,95,chat.completion,2,20240520120310
7,U3,1,Just getting my vitamins in at the school cant...,U320240520120311,"So, I'm just grabbing some quinoa, chickpeas, ...","In this conversational style of person X, the...","tst\_sentence: ""So, I'm just grabbing some qui...",\n[INST]You are an expert in text style transf...,mistral-small,323,145,chat.completion,2,20240520120311
8,U4,0,I’m all about that food. I usually kick off th...,U420240520120313,"Y'know, I'm a real foodie at heart. Gotta star...",The conversational style of person X includes...,"tst\_sentence: ""Y'know, I'm a real foodie at h...",\n[INST]You are an expert in text style transf...,mistral-small,329,149,chat.completion,2,20240520120313
9,U4,1,Just getting my vitamins in at the school cant...,U420240520120315,Gettin' my vitamins at the school cafeteria to...,"In this conversational style of person X, I i...","tst\_sentence: ""Gettin' my vitamins at the sch...",\n[INST]You are an expert in text style transf...,mistral-small,333,153,chat.completion,2,20240520120315


In [167]:
df_eval = p.llm_evl(df_llm_tst_final)

In [7]:
df_eval = pd.read_csv('f8_llm_evaluation_data/eval_20240520173356.csv')

In [10]:
df_eval_final = p.postprocess_llm_evl(df_eval)

Score: 2_somewhat concise/descriptive

Explanation: The text provides some descriptive elements, such as the mention of specific food items (quinoa, chickpeas, soy chicken) and the positive attitude towards the upcoming math exam. However, it is still concise and lacks detailed descriptions that could create a vivid image or evoke stronger emotions.


In [11]:
df_eval_final

,tst_id,tst_sentence,eval_promptID,eval_score_formality,eval_explanation_formality,eval_score_descriptiveness,eval_explanation_descriptiveness,eval_score_emotionality,eval_explanation_emotionality,eval_score_sentiment,eval_explanation_sentiment,eval_score_fluency,eval_score_comprehensibility,eval_explanation_fluency_comprehensibility
0,U020240520120253,"I'm totally into food, you know? I start my we...",4,1,The text contains multiple signs of informalit...,2,The text provides some descriptive elements su...,3,The text is predominantly emotional with the u...,3,The text expresses a positive sentiment toward...,2,3,The text is written in English and uses proper...
1,U020240520120254,So I'm just getting my vitamins at the school ...,4,1,The text contains multiple signs of informali...,1,The text primarily focuses on the speaker's ac...,2,"The text contains some emotional elements, suc...",3,The text has a positive sentiment due to the m...,2,3,"The text is written entirely in English, so it..."
2,U120240520120258,"Got to have my food fix, you know? Startin' th...",4,1,The text contains multiple elements that indic...,2,"The text contains some descriptive elements, s...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm and positivity t...,2,3,"The text is written in English, so it gets a s..."
3,U120240520120259,"No worries about my math exam, I'm good to go!...",4,2,This text contains some elements of informalit...,2,None,3,"The text is predominantly emotional, as it exp...",3,The text expresses a positive attitude towards...,2,3,"The text is written entirely in English, so i..."
4,U220240520120304,You know what I'm totally into? Food! I start ...,4,1,The text contains multiple signs of informali...,2,"The text contains some descriptive elements, s...",3,"The text is predominantly emotional, as it con...",3,The text expresses excitement and enthusiasm a...,2,3,"The text is written in English, so it gets a s..."
5,U220240520120306,"Hey, just grabbing some grub at the school caf...",4,1,The text contains several features that are ch...,2,"The text contains some descriptive elements, s...",2,"The text contains some emotional elements, suc...",3,The text has a positive sentiment due to the m...,2,3,The text is written in English and uses proper...
6,U320240520120310,"Yo, got to get my weekly dose of that vegan go...",4,1,The text contains several informal elements s...,2,"The text contains some descriptive elements, s...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm and excitement a...,2,2,"The text is written in English, but the inform..."
7,U320240520120311,"So, I'm just grabbing some quinoa, chickpeas, ...",4,1,The text contains several features that indica...,1,"The text is predominantly concise, providing m...",3,"The text is predominantly emotional, as it con...",3,The text has a positive sentiment. The speaker...,2,3,"The text is written entirely in English, so th..."
8,U420240520120313,"Y'know, I'm a real foodie at heart. Gotta star...",4,2,The text contains some informal elements such ...,2,The text provides some descriptive elements ab...,3,"The text is predominantly emotional, as it use...",3,The text has a positive sentiment. The speaker...,2,3,The text is written in English and uses proper...
9,U420240520120315,Gettin' my vitamins at the school cafeteria to...,4,1,The text contains multiple instances of non-st...,2,"The text has some descriptive elements, such a...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm about eating hea...,2,2,"The text is written in English, so the score f..."


In [3]:
#count none in dataframe
df_eval_final['eval_score_formality'].describe()

NameError: name 'df_eval_final' is not defined

In [183]:
df_eval_final

,tst_id,tst_sentence,eval_promptID,eval_score_formality,eval_explanation_formality,eval_score_descriptiveness,eval_explanation_descriptiveness,eval_score_emotionality,eval_explanation_emotionality,eval_score_sentiment,eval_explanation_sentiment,eval_score_fluency,eval_score_comprehensibility,eval_explanation_fluency_comprehensibility
0,U020240520120253,"I'm totally into food, you know? I start my we...",4,1,The text contains multiple signs of informalit...,2,The text provides some descriptive elements su...,3,The text is predominantly emotional with the u...,3,The text expresses a positive sentiment toward...,2,3,The text is written in English and uses proper...
1,U020240520120254,So I'm just getting my vitamins at the school ...,4,1,The text contains multiple signs of informali...,1,The text primarily focuses on the speaker's ac...,2,"The text contains some emotional elements, suc...",3,The text has a positive sentiment due to the m...,2,3,"The text is written entirely in English, so it..."
2,U120240520120258,"Got to have my food fix, you know? Startin' th...",4,1,The text contains multiple elements that indic...,2,"The text contains some descriptive elements, s...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm and positivity t...,2,3,"The text is written in English, so it gets a s..."
3,U120240520120259,"No worries about my math exam, I'm good to go!...",4,2,This text contains some elements of informalit...,2,None,3,"The text is predominantly emotional, as it exp...",3,The text expresses a positive attitude towards...,2,3,"The text is written entirely in English, so i..."
4,U220240520120304,You know what I'm totally into? Food! I start ...,4,1,The text contains multiple signs of informali...,2,"The text contains some descriptive elements, s...",3,"The text is predominantly emotional, as it con...",3,The text expresses excitement and enthusiasm a...,2,3,"The text is written in English, so it gets a s..."
5,U220240520120306,"Hey, just grabbing some grub at the school caf...",4,1,The text contains several features that are ch...,2,"The text contains some descriptive elements, s...",2,"The text contains some emotional elements, suc...",3,The text has a positive sentiment due to the m...,2,3,The text is written in English and uses proper...
6,U320240520120310,"Yo, got to get my weekly dose of that vegan go...",4,1,The text contains several informal elements s...,2,"The text contains some descriptive elements, s...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm and excitement a...,2,2,"The text is written in English, but the inform..."
7,U320240520120311,"So, I'm just grabbing some quinoa, chickpeas, ...",4,1,The text contains several features that indica...,1,"The text is predominantly concise, providing m...",3,"The text is predominantly emotional, as it con...",3,The text has a positive sentiment. The speaker...,2,3,"The text is written entirely in English, so th..."
8,U420240520120313,"Y'know, I'm a real foodie at heart. Gotta star...",4,2,The text contains some informal elements such ...,2,The text provides some descriptive elements ab...,3,"The text is predominantly emotional, as it use...",3,The text has a positive sentiment. The speaker...,2,3,The text is written in English and uses proper...
9,U420240520120315,Gettin' my vitamins at the school cafeteria to...,4,1,The text contains multiple instances of non-st...,2,"The text has some descriptive elements, such a...",3,The text is predominantly emotional due to the...,3,The text expresses enthusiasm about eating hea...,2,2,"The text is written in English, so the score f..."
